In [1]:
import numpy as np
import pandas as pd
import heapq as hp
import multiprocessing as mp
from time import sleep

In [56]:
class StockMarket:
    # Потом сюда прикручу получение данных онлайн 
    bids = []
    asks = []
    stored_number_of_currency = 5
    currency_pair = 'btcusdt'
    data = []
    def current_price(self):
        return (-self.bids[0][0] + self.asks[0][0]) / 2
    
    
    # Можно сделать def run() которая начаинает процесс - с таймаутом data['time'][i] - data['time'][i-1]
    # обновляет стакан и умеет обрабатывать наши запросы на покупку-продажу. Это работает на multiprocessing
    
    
    def add_data(self, idx):
        current_amount = 0
        index = 1
        # у bid'ов самая лучшая цена - наибольшая -> в очереди на минимумах храним отрицательное значение.
        # У ask'ов наоборот
        while index < 20 and current_amount < self.stored_number_of_currency:
            hp.heappush(self.bids, (-self.data[f'depth_bid_price_{index}'][idx],
                               self.data[f'depth_bid_quantity_{index}'][idx]))
            current_amount += self.data[f'depth_bid_quantity_{index}'][idx]
            index += 1
        current_amount = 0
        index = 1
        while index < 20 and current_amount < self.stored_number_of_currency:
            hp.heappush(self.asks, (self.data[f'depth_ask_price_{index}'][idx],
                               self.data[f'depth_ask_quantity_{index}'][idx]))
            current_amount += self.data[f'depth_ask_quantity_{index}'][idx]
            index += 1
        
    
    def make_buy_order(self, amount):
        if amount > self.stored_number_of_currency:
            return -1
        spent = 0
        while amount > 0:
            if self.asks[0][1] > amount:
                self.asks[0][1] -= amount
                spent += self.asks[0][0] * amount
                amount = 0
            else:
                amount -= self.asks[0][1]
                spent += self.asks[0][1] * self.asks[0][0]
                hp.heappop(self.asks)
        return spent

    
    def make_sell_order(self, amount):
        if amount > self.stored_number_of_currency:
            return -1
        capital = 0
        while amount > 0:
            if self.bids[0][1] > amount:
                self.bids[0][1] -= amount
                capital += self.bids[0][0] * amount
                amount = 0
            else:
                amount -= self.bids[0][1]
                capital += self.bids[0][0] * self.bids[0][1]
                hp.heappop(self.bids)
        return capital
               
        
    def predict_price_movement(self, goes_up):
        # прогнозирование движения цены на следующей итерации стакана
        return
        
    # С помощью библиотеки time можно измерить сколько в среднем требуется времени на запись следующего куска 
    # стакана и каждый раз делать sleep(time_cur - time_prev - execution_time) чтобы все работало как надо
    
    def run(self, idx = 0, limit = None):
        if limit == None or limit > self.data.shape[0]:
            limit = self.data.shape[0]
        while idx < limit:
            self.asks.clear()
            self.bids.clear()
            self.add_data(idx)
            idx += 1
            sleep(self.data['time'][idx] - self.data['time'][idx - 1] - 0.02)
            print(self.current_price())

        # ну все, теперь осталось запустить вне класса параллельно процесс run и то, что отправляет запросы к 
        # нашему классу(если правильно работает память, на что я очень надеюсь, иначе shared_memory...)
    
    def __init__ (self, table, cur_pair = 'btcusdt'):
        self.currency_pair = cur_pair
        self.data = table[table['currency_pair']==cur_pair]
        self.data.index = range(self.data.shape[0])

In [6]:
r = open('names.txt', 'r')
name = eval(r.read())
r.close
table = pd.read_csv('/home/ivan/Prog/Python/Distonx/distonx.csv', names=name)

In [7]:
table.head()

,id,time,currency_pair,kline_trade_number,kline_open_price,kline_close_price,kline_high_price,kline_low_price,kline_base_volume,kline_quote_volume,...,depth_ask_quantity_11,depth_ask_quantity_12,depth_ask_quantity_13,depth_ask_quantity_14,depth_ask_quantity_15,depth_ask_quantity_16,depth_ask_quantity_17,depth_ask_quantity_18,depth_ask_quantity_19,depth_ask_quantity_20
0,1,1.581428e+09,bnbusdt,90,25.090500,25.072600,25.090500,25.059400,1168.770000,29310.907722,...,153.73000,13.700000,5.850000,6.92000,555.75000,24.130000,102.650000,6.920000,119.630000,13.840000
1,2,1.581428e+09,ltcbtc,37,0.007513,0.007514,0.007517,0.007513,76.600000,0.575639,...,181.73000,293.310000,0.260000,173.24000,4.95000,40.640000,62.730000,20.120000,33.220000,43.880000
2,3,1.581428e+09,ethbtc,209,0.022668,0.022672,0.022684,0.022665,802.450000,18.192120,...,22.89600,1.400000,1.455000,3.62500,5.82800,1.426000,1.710000,1.400000,114.480000,97.466000
3,4,1.581428e+09,btcusdt,637,9899.420000,9903.110000,9904.760000,9895.080000,53.008219,524722.027914,...,0.60000,0.020191,0.131554,1.20000,0.20000,0.036433,0.005157,0.929697,0.017545,0.619148
4,5,1.581428e+09,ethusdt,211,224.410000,224.520000,224.550000,224.380000,684.376270,153590.034043,...,27.70203,17.012800,10.499550,11.01988,157.30799,98.785130,12.290570,38.555050,6.381370,8.556130


In [68]:
btc_data = table[table['currency_pair']=='btcusdt']

In [100]:
btc_data[['depth_bid_price_1', 'depth_bid_price_3', 'depth_ask_price_1', 'depth_ask_price_5']]

,depth_bid_price_1,depth_bid_price_3,depth_ask_price_1,depth_ask_price_5
3,9903.51,9902.52,9903.93,9904.92
14,9905.00,9903.55,9905.02,9905.49
25,9905.00,9903.55,9905.02,9905.62
36,9899.16,9898.39,9899.77,9900.54
47,9898.90,9898.09,9900.48,9901.54
...,...,...,...,...
499953,10270.93,10270.91,10271.00,10272.06
499964,10271.43,10271.38,10271.91,10272.32
499975,10271.83,10271.76,10272.00,10272.90
499986,10271.56,10271.49,10272.59,10272.84


In [133]:
btc_data['time'].index=range(btc_data['time'].size)

In [125]:
print(min(btc_data['time'][i + 1] - btc_data['time'][i] for i in range(45454)))
print(max(btc_data['time'][i + 1] - btc_data['time'][i] for i in range(45454)))

1.0925037860870361
2.99485445022583


In [57]:
bitcoin = StockMarket(table=table, cur_pair='btcusdt')

In [64]:
bitcoin.add_data(20)

In [58]:
def ask_prices():
    print(bitcoin.current_price())
    

def queries(number=10):
    for i in range(number):
        ask_prices()
        sleep(1.2)
        

In [65]:
if __name__ == '__main__': 
    market = mp.Process(target=bitcoin.run)
    model = mp.Process(target=queries)
    market.start()
    model.start()
    
    
    model.join()
    market.terminate()

9894.585
9903.720000000001
9894.585
9905.01
9894.585
9905.01
9894.585
9899.465
9894.585
9899.689999999999
9894.585
9899.415
9894.585
9899.630000000001
9894.585
9899.45
9894.585
9899.185000000001
9894.585
9899.19


In [54]:
(bitcoin.asks[0][0] - bitcoin.bids[0][0])/2

9897.075